<a href="https://colab.research.google.com/github/dudany/deep_music_gen/blob/master/RNN_music_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install dropbox
import dropbox
import numpy as np
import pandas as pd
import pydub
import os
from collections import namedtuple
from keras.layers import Dense, LSTM, LeakyReLU
from keras.models import Sequential, load_model
from scipy.io.wavfile import read, write


**If you want to train your model on mp3 files, the following lines of code will do the trick.**


In [ ]:

# converting mp3 file to wav file
# sound = pydub.AudioSegment.from_mp3("Numb_piano.mp3")
# sound.export("Numb.wav", format="wav")
# sound = pydub.AudioSegment.from_mp3("Eminem.mp3")
# sound.export("Eminem.wav", format="wav")



as we Have .wav data set we'll not to the conversion

In [ ]:
# loading the wav files
MusicTup = namedtuple('MusicTup', ['rate', 'music'])
auth_dbx_key = 'sl.Aikv_wak-0pR9Ay9jG1dwqf_HkTIrjs8Za8EXWmNW-952pr72M3zCGbi4W4KwOEohsY70hiGccCtK8hwI3ejHtx8fGrGiFy6sjqKRsNhus8p9sJNWIXAYJ1ISIYPReUgis8R67o'
data_path = 'https://www.dropbox.com/home/Apps/music_gen'
data_list = []

dbx = dropbox.Dropbox(auth_dbx_key)

with open("Prime_Numbers.txt", "wb") as f:
    metadata, res = dbx.files_download(path=data_path)
    f.write(res.content)



for f in os.listdir(data_path):
  path2f = os.path.join(data_path, f)
  if os.path.isfile(path2f):
    rate, music = read(path2f)
    music = pd.DataFrame(music)
    data_list.append(MusicTup(rate,music))


ValidationError: ignored

We can clearly see that the music dataframe has two columns, namely the two channels of our song and it is of integer type. What we are going to do is train two models on the two channels and make predictions separately for both the channels and then concatenate the results to generate a piece of music.


In [ ]:
# function to create training data by shifting the music data 
def create_train_dataset(df, look_back, train=True):
    dataX1, dataX2 , dataY1 , dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        if train:
            dataY1.append(df.iloc[i + look_back, 0])
            dataY2.append(df.iloc[i + look_back, 1])
    if train:
        return np.array(dataX1), np.array(dataX2), np.array(dataY1), np.array(dataY2)
    else:
        return np.array(dataX1), np.array(dataX2)

X1, X2, y1, y2 = create_train_dataset(pd.concat([music1.iloc[0:160000, :],music2.iloc[0:160000, :]], axis=0), look_back=3, train=True)

NameError: ignored